In [1]:
import os

In [2]:
os.getcwd()

'c:\\Users\\kafub\\Dropbox\\PC\\Desktop\\Python_and ML_BootCamp_from _Udemy\\DL\\chickenDiseaseClassification\\research'

In [3]:
os.chdir("../")

In [4]:
os.getcwd()

'c:\\Users\\kafub\\Dropbox\\PC\\Desktop\\Python_and ML_BootCamp_from _Udemy\\DL\\chickenDiseaseClassification'

In [28]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [29]:
import src.CNNClassifier.utils.common1 as common1
print(dir(common1))

['Any', 'BoxValueError', 'ConfigBox', 'Path', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'base64', 'create_directories', 'decodeImage', 'encodeImageIntoBase64', 'ensure_annotations', 'get_size', 'joblib', 'json', 'load_bin', 'load_json', 'logger', 'os', 'read_yaml', 'save_bin', 'save_json', 'yaml']


In [13]:
import src.CNNClassifier.constants as constants
print(dir(constants))

['CONFIG_FILE_PATH', 'PARAMS_FILE_PATH', 'Path', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__']


In [30]:
from src.CNNClassifier.constants import *
from src.CNNClassifier.utils.common1 import read_yaml, create_directories

In [31]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

In [32]:
import os
import urllib.request as request
import zipfile
from src.CNNClassifier.logging import logger
from src.CNNClassifier.utils.common1 import get_size


In [33]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [35]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-09-17 13:33:04,196]: INFO:common1: yaml file: config\config.yaml loaded successfully
[2024-09-17 13:33:04,196]: INFO:common1: yaml file: params.yaml loaded successfully
[2024-09-17 13:33:04,202]: INFO:common1: created directory at: artifacts
[2024-09-17 13:33:04,202]: INFO:common1: created directory at: artifacts/data_ingestion
[2024-09-17 13:33:05,542]: INFO:4037744573: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 4101113
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "d9f898b60916fc9c6022f41ba5f324079c7cbe19e1b5dfca51b7092f4e533549"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 432D:1114:B2513E:BA6F0C:66E9E781
Accept-Ranges: bytes
Date: Tue, 17 Sep 2024 20:33:05 GMT
Via: 1.1 varnish
X-Served-By: cache-pao-kpao1770044-PAO
X-Cac